In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import os 
import timeit
import math
import time
import sparse
import dask
import dask.array as da
from dask.diagnostics import ProgressBar
from glob import glob
from os import path
import pickle
from tqdm import tqdm

from ll_Balltree import *
%run -i 'll_Balltree.py'

In [2]:
outputDir = 'data/posterior_computation_data/'

In [3]:
# Load domain_cell_tree
fileObj = open(outputDir + 'output_dict.obj', 'rb')
output = pickle.load(fileObj)
fileObj.close()

In [4]:
n_cell_beaching = output['n_cell_beaching']
n_cell_source = output['n_cell_source']
n_window_beaching = output['n_window_beaching'] 
n_window_source = output['n_window_source']

particle_count = output['particle_count']

beaching_cells = output['beaching_cells'] 
beaching_cell_tree = output['beaching_cell_tree']

source_cell_mask = output['source_cell_mask']
source_cells = output['source_cells']
source_cell_tree = output['source_cell_tree']

beaching_windows = output['beaching_windows']
source_windows = output['source_windows']
d = output['d']
beaching_ym_mat = output['beaching_ym_mat']
source_ym_mat = output['source_ym_mat']

n_cell_beaching,n_cell_source,n_window_beaching, n_window_source

(188, 114024, 30, 36)

In [5]:
ds = xr.open_zarr(outputDir + 'nn_post_reshaped.zarr')

In [6]:
with ProgressBar():
    n_post_norm = np.sum(ds.l_n_post, axis = 0) +  np.sum(ds.r_n_post, axis = 0) + np.sum(ds.f_n_post, axis = 0)

    l_n_post  = np.nan_to_num(ds.l_n_post / n_post_norm)
    r_n_post  = np.nan_to_num(ds.r_n_post / n_post_norm)
    f_n_post  = np.nan_to_num(ds.f_n_post / n_post_norm)

[#                                       ] | 3% Completed | 4.00 s ms

/home/dpojunas/.conda/envs/py3_parcels/lib/python3.10/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed | 124.39 s
[#                                       ] | 3% Completed | 3.99 s ms

/home/dpojunas/.conda/envs/py3_parcels/lib/python3.10/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed | 124.48 s
[#                                       ] | 3% Completed | 3.99 s ms

/home/dpojunas/.conda/envs/py3_parcels/lib/python3.10/site-packages/dask/core.py:121: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


[########################################] | 100% Completed | 125.60 s


In [8]:
n_post = xr.Dataset(
    {
        'l_n_post': (['dsource_windows_cells', 'beaching_window_cells' ], l_n_post),
        'r_n_post': (['dsource_windows_cells', 'beaching_window_cells' ], r_n_post),
        'f_n_post': (['dsource_windows_cells', 'beaching_window_cells' ], f_n_post),
    },
    coords={
        'dsource_windows_cells': np.arange(n_cell_source*(d+1)),
        'beaching_window_cells': np.arange((n_cell_beaching*n_window_beaching)),
    },
)

TypeError: Using a DataArray object to construct a variable is ambiguous, please extract the data using the .data property.

In [8]:
chunksize = {'dsource_windows_cells': n_cell_source,
             'beaching_window_cells': n_cell_beaching}

print('re-chunking')
tic=time.time()
n_post = n_post.chunk(chunksize)
print('   done in',time.time()-tic)

re-chunking
   done in 127.65097260475159


In [9]:
delayed_obj = n_post.to_zarr(outputDir + "n_post.zarr", compute=False)

with ProgressBar():
    results = delayed_obj.compute()

[########################################] | 100% Completed | 169.72 s
